In [2]:
import pandas as pd
import numpy as np

# Model selection libraries
from sklearn.model_selection import train_test_split

# Model Evaluation Libraries
from sklearn.metrics import accuracy_score

# Import Tensor Flow and keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_hub as hub
import tensorflow_text

2023-03-29 16:42:46.285021: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# Import warnings and supress them
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Configure Pandas to show all columns / rows
pd.options.display.max_columns = 2000
pd.options.display.max_rows = 2000
# Set column max width larger
pd.set_option('display.max_colwidth', 200)

In [5]:
# Load data from phishing email csv
email_df = pd.read_csv('./data/phishing_all_data_clean_4.csv')

In [6]:
X = email_df.drop(columns=['phishing'])
y = email_df.loc[:, ['phishing']]

In [7]:
X_remainder, X_test, y_remainder, y_test = train_test_split(
    X,
    y,
    stratify=y,
    random_state=1337,
    test_size=0.2,
)
X_remainder.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
y_remainder.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

In [8]:
X_train, X_validation, y_train, y_validation = train_test_split(
    X_remainder,
    y_remainder,
    stratify=y_remainder,
    random_state=1337,
    test_size=0.2,
)
X_train.reset_index(drop=True, inplace=True)
X_validation.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
y_validation.reset_index(drop=True, inplace=True)

In [9]:
preprocessor = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4",
    trainable=False)
# encoder_inputs = preprocessor(text_input)
# outputs = encoder(encoder_inputs)
# pooled_output = outputs["pooled_output"]      # [batch_size, 768].

In [10]:
text_input = layers.Input(shape=(), dtype=tf.string)
encoder_inputs = preprocessor(text_input)
outputs = encoder(encoder_inputs)

pooled_output = outputs["pooled_output"]      # [batch_size, 768].
sequence_output = outputs["sequence_output"]  # [batch_size, seq_length, 768].
relu_layer = layers.Dense(128, activation='relu')(pooled_output)
output = layers.Dense(1, activation='sigmoid')(relu_layer)

In [11]:
bert_model = tf.keras.Model(inputs=text_input, outputs=output)

In [13]:
bert_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_mask': (Non  0           ['input_1[0][0]']                
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

In [17]:
bert_model.compile(
    # Optimizer
    optimizer=keras.optimizers.Adam(),
    # Loss function to minimize
    loss=keras.losses.BinaryCrossentropy(),
    # Metric used to evaluate model
    metrics=[keras.metrics.BinaryAccuracy(), keras.metrics.Recall()]
)

In [20]:
history = bert_model.fit(X_remainder['content'], y_remainder, epochs=20, verbose=1, validation_split=0.2)

Epoch 1/5
248/248 [==============================] - 2133s 9s/step - loss: 0.1412 - binary_accuracy: 0.9509
Epoch 2/5
248/248 [==============================] - 1053s 4s/step - loss: 0.1318 - binary_accuracy: 0.9521
Epoch 3/5
248/248 [==============================] - 11396s 46s/step - loss: 0.1254 - binary_accuracy: 0.9549
Epoch 4/5
248/248 [==============================] - 3075s 12s/step - loss: 0.1383 - binary_accuracy: 0.9514
Epoch 5/5
248/248 [==============================] - 9907s 40s/step - loss: 0.1153 - binary_accuracy: 0.9582


In [23]:
y_val_pred = bert_model.predict(X_validation['content'])

62/62 [==============================] - 270s 4s/step


In [26]:
y_val_bin_pred = np.where(y_val_pred >= 0.5, 1, 0)

In [27]:
accuracy_score(y_validation, y_val_bin_pred)

0.9647177419354839

In [17]:
bert_model.predict(["""

"""])

1/1 [==============================] - 0s 229ms/step


array([[0.29253405]], dtype=float32)